<a href="https://colab.research.google.com/github/Nawaz8179/Nawaz8179/blob/main/TEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyTelegramBotAPI torch torchvision flask-ngrok pyngrok
!pip install pyngrok



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.9/242.9 kB 3.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
from pyngrok import ngrok

authtoken = "2glzebBX7jL9jBlVlC8qk9IDAW0_6gdPqZZn33gW7skUpP1fc"
ngrok.set_auth_token(authtoken)


In [8]:
import threading
import telebot
import torch
import torchvision
from torchvision import transforms, datasets
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from flask import Flask, request
from pyngrok import ngrok
import requests
from PIL import Image

# Set your ngrok authtoken
authtoken = "2glzebBX7jL9jBlVlC8qk9IDAW0_6gdPqZZn33gW7skUpP1fc"
ngrok.set_auth_token(authtoken)

TELEGRAM_API_KEY = '7142689384:AAEW5xr0ESiFr77X8933I1uhPQrDkjWQ-Ug'
bot = telebot.TeleBot(TELEGRAM_API_KEY)

transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(2):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
print('Finished Training')

app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    image_file = request.files['image']
    image = Image.open(image_file).convert('RGB')
    image = transform(image).unsqueeze(0)
    outputs = net(image)
    _, predicted = torch.max(outputs, 1)
    prediction = classes[predicted[0]]
    return prediction

def start_ngrok():
    tunnel_url = ngrok.connect(5000).public_url
    print(f" * ngrok tunnel {tunnel_url} -> http://127.0.0.1:5000")
    return tunnel_url

public_url = start_ngrok()

def run_flask():
    app.run(port=5000)

flask_thread = threading.Thread(target=run_flask)
flask_thread.start()


Files already downloaded and verified
[1,  2000] loss: 2.218
[1,  4000] loss: 1.826
[1,  6000] loss: 1.656
[1,  8000] loss: 1.582
[1, 10000] loss: 1.526
[1, 12000] loss: 1.459
[2,  2000] loss: 1.413
[2,  4000] loss: 1.373
[2,  6000] loss: 1.366
[2,  8000] loss: 1.349
[2, 10000] loss: 1.312
[2, 12000] loss: 1.280
Finished Training
 * ngrok tunnel https://c597-35-247-102-162.ngrok-free.app -> http://127.0.0.1:5000
 * Serving Flask app '__main__'
 * Debug mode: off


In [ ]:
import telebot
import requests


TELEGRAM_API_KEY = '7142689384:AAEW5xr0ESiFr77X8933I1uhPQrDkjWQ-Ug'
bot = telebot.TeleBot(TELEGRAM_API_KEY)

@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
    user_name = message.from_user.first_name
    creator_name = "Nawaz Khan"  # Replace with your name
    bot.reply_to(message, f"Hello {user_name}! 👋 I'm your friendly assistant, created by {creator_name}. How can I assist you today?")


@bot.message_handler(content_types=['photo'])
def handle_photo(message):

    file_info = bot.get_file(message.photo[-1].file_id)
    downloaded_file = bot.download_file(file_info.file_path)


    with open('image.jpg', 'wb') as new_file:
        new_file.write(downloaded_file)


    response = requests.post(f"{public_url}/predict", files={'image': open('image.jpg', 'rb')})
    bot.reply_to(message, f"Predicted class: {response.text}")


bot.polling()
